In [ ]:
import os
import time
import json
import pytz
import random
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone

In [ ]:
ids = []
removed = 0
for i in range(1, 12):
    url = "https://sospo.myswitzerland.com/lakesides-swimming-pools/lake.river/?p={}".format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find_all("table", class_="FilterGridTable--table")[0]
    for row in table.find_all('tr')[1:]:
        if len(row.find_all("div", class_="FilterGridTable--title")) == 1:
            keys = [li.get_text(strip=True) for li in row.find_all('li', class_='QuickFacts--content')]
            values = [li.get_text(strip=True) for li in row.find_all('li', class_='QuickFacts--value')]
            properties = dict(zip(keys, values))
            if properties["Lake"] != "—" or properties["River"] != "—":
                link = row.find("a", class_="FilterGridTable--link").get('href')
                p = requests.get(link)
                s = BeautifulSoup(p.content, "html.parser")
                if s.find("div", class_="QuickFactsWidget--info is-open") != None:
                    date = datetime.strptime(s.find("div", class_="QuickFactsWidget--info is-open").get_text(strip=True).split(": ", 1)[1], "%d.%m.%Y, %H:%M")
                    if (date > datetime.now() - timedelta(days=30)):
                        ids.append(link.split("/")[-2])
                    else:
                        removed = removed + 1
                time.sleep(random.uniform(0, 1))
with open("myswitzerland.json", 'w') as f:
    json.dump(ids, f)